In [22]:
import pandas as pd
import os

from os.path import dirname

root_path = dirname(os.getcwd())

pd.set_option('display.max_columns', None)
data_dir = root_path + '/data/datasets/original/'
data_dir_processed = root_path + '/data/datasets/processed/'

print(root_path, data_dir, data_dir_processed, sep='\n')

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/


In [23]:
dataset = 'BPI_Challenge_2012_A'
filename = data_dir + dataset + '.csv'

raw_data = pd.read_csv(filename, index_col=False)

raw_data = raw_data.drop(columns='Unnamed: 0')

# print(raw_data.head())

columns_key = list(raw_data.columns)
print(columns_key)



['org:resource', 'lifecycle:transition', 'concept:name', 'time:timestamp', 'case:REG_DATE', 'case:concept:name', 'case:AMOUNT_REQ']


In [24]:
raw_data.head()

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,10862,COMPLETE,A_ACCEPTED,2011-10-01 09:42:43.308000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
4,10862,COMPLETE,A_FINALIZED,2011-10-01 09:45:09.243000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000


In [27]:
tab_all = raw_data.rename(columns={'case:concept:name' : 'CaseID', 'concept:name': 'Activity'})
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,10862,COMPLETE,A_ACCEPTED,2011-10-01 09:42:43.308000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000
4,10862,COMPLETE,A_FINALIZED,2011-10-01 09:45:09.243000+00:00,2011-10-01 00:38:44.546000+02:00,173688,20000


In [31]:
tab_all['time:timestamp'] = [x.split('.')[0] for x in tab_all['time:timestamp']]
tab_all['case:REG_DATE'] = [x.split('.')[0] for x in tab_all['case:REG_DATE']]
tab_all['time:timestamp'] = tab_all['time:timestamp'].str.replace("-", "/")
tab_all['time:timestamp'] = tab_all['time:timestamp'].str.split('+', expand = True)[0]
tab_all['case:REG_DATE'] = tab_all['case:REG_DATE'].str.replace("-", "/")
tab_all['case:REG_DATE'] = tab_all['case:REG_DATE'].str.split('+', expand = True)[0]
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011/09/30 22:38:44,2011/10/01 00:38:44,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011/09/30 22:38:44,2011/10/01 00:38:44,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011/09/30 22:39:37,2011/10/01 00:38:44,173688,20000
3,10862,COMPLETE,A_ACCEPTED,2011/10/01 09:42:43,2011/10/01 00:38:44,173688,20000
4,10862,COMPLETE,A_FINALIZED,2011/10/01 09:45:09,2011/10/01 00:38:44,173688,20000


In [32]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

In [39]:
split_ratio = 2/3

first_act_tab = tab_all.groupby('CaseID').first().sort_values('time:timestamp').reset_index()
first_act_tab = first_act_tab[~first_act_tab.duplicated(subset=["CaseID", "Activity"], keep = "first")]
first_act_tab = first_act_tab.reset_index(drop = True)

list_train_valid_cases = list(first_act_tab[: int(split_ratio*len(first_act_tab))]["CaseID"].unique())

list_train_cases = list_train_valid_cases[:int(len(list_train_valid_cases)*0.8)]
tab_train = tab_all[tab_all["CaseID"].isin(list_train_cases)].reset_index(drop = True)
tab_train.to_csv(data_dir_processed+ f"{dataset}_processed_train.csv", index = False)

list_valid_cases = list_train_valid_cases[int(len(list_train_valid_cases)*0.8):]
tab_valid = tab_all[tab_all["CaseID"].isin(list_valid_cases)].reset_index(drop = True)
tab_valid.to_csv(data_dir_processed+f"{dataset}_processed_valid.csv", index = False)

list_test_cases = list(first_act_tab[int(split_ratio*len(first_act_tab)):]["CaseID"].unique())
tab_test = tab_all[tab_all["CaseID"].isin(list_test_cases)].reset_index(drop = True)
tab_test.to_csv(data_dir_processed+ f"{dataset}_processed_test.csv", index = False)


### Prepare the graphs

In [25]:
from torch_geometric.data import HeteroData

In [43]:
case_ids = list(set(tab_valid['CaseID']))

In [ ]:
# Fare dizionario colonna X is_static() per ogni colonna

In [90]:
trace_ex = tab_valid.query(f'CaseID == {case_ids[1]}').reset_index().drop(columns='index')
trace_ex = trace_ex.drop(columns="CaseID")
trace_ex

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011/12/29 22:22:07,2011/12/29 23:22:07,2500
1,112,COMPLETE,A_PARTLYSUBMITTED,2011/12/29 22:22:07,2011/12/29 23:22:07,2500
2,112,COMPLETE,A_PREACCEPTED,2011/12/29 22:22:42,2011/12/29 23:22:07,2500
3,11202,COMPLETE,A_DECLINED,2011/12/30 17:46:21,2011/12/29 23:22:07,2500


In [91]:
columns_key = list(trace_ex.columns)
columns_key

['org:resource',
 'lifecycle:transition',
 'Activity',
 'time:timestamp',
 'case:REG_DATE',
 'case:AMOUNT_REQ']

In [103]:
def feature_dynamic(values, key, difference):
    res = {}
    for i in range(len(values)):
        for j in range(i+1, len(values)):
            res[(f'{key}_{i}', f'{key}_{j}')] =  difference(values[j], values[i])
    return res
            

In [105]:
from pprint import pprint as print
print(feature_dynamic(trace_ex['org:resource'], 'org:resource', lambda x,y : x-y))

{('org:resource_0', 'org:resource_1'): 0,
 ('org:resource_0', 'org:resource_2'): 0,
 ('org:resource_0', 'org:resource_3'): 11090,
 ('org:resource_1', 'org:resource_2'): 0,
 ('org:resource_1', 'org:resource_3'): 11090,
 ('org:resource_2', 'org:resource_3'): 11090}


In [95]:
from utils import is_static


columns_static = {c : is_static(trace_ex[c]) for c in columns_key}
columns_static

{'org:resource': False,
 'lifecycle:transition': True,
 'Activity': False,
 'time:timestamp': False,
 'case:REG_DATE': True,
 'case:AMOUNT_REQ': True}

In [ ]:
import time 
import datetime

In [81]:
trace_ex = trace_ex.values

['org:resource',
 'lifecycle:transition',
 'Activity',
 'time:timestamp',
 'case:REG_DATE',
 'CaseID',
 'case:AMOUNT_REQ']

In [77]:
list(trace_ex.values)

[array([112, 'COMPLETE', 'A_SUBMITTED', '2011/12/29 22:22:07',
        '2011/12/29 23:22:07', 196611, 2500], dtype=object),
 array([112, 'COMPLETE', 'A_PARTLYSUBMITTED', '2011/12/29 22:22:07',
        '2011/12/29 23:22:07', 196611, 2500], dtype=object),
 array([112, 'COMPLETE', 'A_PREACCEPTED', '2011/12/29 22:22:42',
        '2011/12/29 23:22:07', 196611, 2500], dtype=object),
 array([11202, 'COMPLETE', 'A_DECLINED', '2011/12/30 17:46:21',
        '2011/12/29 23:22:07', 196611, 2500], dtype=object)]